In [27]:
dims = rand(1:100, 22);

In [28]:
using BenchmarkTools

function matrix_chain_order(p::Vector{Int}, i::Int, j::Int)
    if i == j
        return 0
    end
    min_cost = typemax(Int)
    for k in i:j-1
        cost = matrix_chain_order(p, i, k) +
               matrix_chain_order(p, k+1, j) +
               p[i] * p[k+1] * p[j+1]
        min_cost = min(min_cost, cost)
    end
    return min_cost
end

# Example usage:
# For matrices A, B, C, D with dimensions:
# A: 30x14, B: 14x7, C: 7x2, D: 2x5,
# define p such that A has dimensions p[1] x p[2], B has p[2] x p[3], etc.
n = length(dims) - 1      # number of matrices
println("Minimum multiplication cost: ", @time matrix_chain_order(dims, 1, n))


  7.215222 seconds
Minimum multiplication cost: 328302


In [29]:
function best_par_memo(v)
    n = length(v)
    memo = Dict{Tuple{Int,Int}, Tuple{Int,Int,Int}}()

    function rec(i, j)
        if haskey(memo, (i, j))
            return memo[(i, j)]
        end

        if j - i == 1
            result = (0, v[i], v[j])
        elseif j - i == 2
            result = (v[i] * v[i+1] * v[j], v[i], v[j])
        else
            best_cost = Inf
            best_result = nothing
            for k in (i+1):(j-1)
                left = rec(i, k)
                right = rec(k, j)
                cost = left[1] + right[1] + v[i] * v[k] * v[j]
                if cost < best_cost
                    best_cost = cost
                    best_result = (best_cost, v[i], v[j])
                end
            end
            result = best_result
        end

        memo[(i, j)] = result
        return result
    end

    return rec(1, n)
end
println("Minimum multiplication cost with memoization: ", @btime best_par_memo(dims)[1])


  676.000 μs (10850 allocations: 280.75 KiB)
Minimum multiplication cost with memoization: 328302
